# Oseen Wake

This notebook deals with the Oseen wake that is responsible for the lifting force and back flipping of air bubbles. The flow field can be expressed in spherical coordinates as follows:

$$
\frac{u_r}{U} = -\frac{a^3 \cos \theta}{2r^3} + \frac{3a^2}{2r^2 Re} \left\{ 1 - \exp\left[ -\frac{rRe}{2a} (1 + \cos\theta) \right] \right\} \\- \frac{3a(1-\cos\theta)}{4r} \exp\left[ -\frac{rRe}{2a} (1 + \cos\theta) \right] 
$$

$$
\frac{u_\theta}{U} = -\frac{a^3\sin\theta}{4r^3} - \frac{3a\sin\theta}{4r}\exp\left[ -\frac{rRe}{2a} (1+\cos\theta) \right]
$$

$$
\frac{u_\varphi}{U} = 0
$$

Since the bubble simulation is formulated in Cartesian coordinates, we convert the above to $u_x$, $u_y$ and $u_z$ using:

$$
u_x = u_r \sin\theta\cos\phi + u_\theta \cos\theta\cos\phi - u_\phi\sin\phi
$$

$$
u_y = u_r \sin\theta\sin\phi + u_\theta \cos\theta\sin\phi + u_\phi\cos\phi
$$

$$
u_z = u_r \cos\theta - u_\theta\sin\theta
$$

where 

$$
\sin\theta = \frac{\sqrt{x^2 + y^2}}{r}
$$

$$
\cos\theta = \frac{z}{r}
$$

$$
\sin\phi = \frac{y}{\sqrt{x^2 + y^2}}
$$

$$
\cos\phi = \frac{x}{\sqrt{x^2 + y^2}}
$$

and 

$$
r = \sqrt{x^2 + y^2 + z^2}
$$

The computation of Oseen wake flow field has been coded in the `Bubble` class. In this notebook, we test the functions associated with this class. 

## 0 Packages

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv
pv.set_jupyter_backend("trame")
from bubble_bouncing.bubble import Bubble

## 1 Velocity field in Cartesian coordinates

Implemented in `Bubble.Oseen_wake()`.

### 1.1 `Oseen_wake_z(self, U, points)`

The flow field around a sphere that's moving in -z direction. This function is a direct coding of the formula in the book by Guazzelli and Morris. One should expect a flow field with radial outward flow in the far field, and radial inward flow in the wake.

In [2]:
a = 6e-4
U = np.array([0, 0, 0.3])
bubble = Bubble(a, U=U)

In [3]:
# define mesh
points = bubble._grid(10)

In [4]:
# flow
flow = bubble.Oseen_wake_z(0.3, points)

In [5]:
mesh = pv.PolyData(points)
mesh["flow"] = flow
glyph = mesh.glyph(orient="flow", scale="flow", factor=0.005)
pl = pv.Plotter()
pl.add_mesh(glyph)
pl.show_axes()
pl.show()

Widget(value='<iframe src="http://localhost:53988/index.html?ui=P_0x172090080_0&reconnect=auto" class="pyvista…

### 1.2 `Oseen_wake(self, points)`

Oseen wake flow field for bubble moving in arbitrary directions. The flow field should rotate corresponding to the input velocity U.

In [6]:
a = 6e-4
U = np.array([0, 0.3, 0.])
bubble = Bubble(a, U=U)
points = bubble._grid()

In [7]:
points

array([[-0.0012    , -0.0012    , -0.0012    ],
       [-0.0012    , -0.0012    , -0.00093333],
       [-0.0012    , -0.0012    , -0.00066667],
       ...,
       [ 0.0012    ,  0.0012    ,  0.00066667],
       [ 0.0012    ,  0.0012    ,  0.00093333],
       [ 0.0012    ,  0.0012    ,  0.0012    ]], shape=(1000, 3))

In [8]:
flow = bubble.Oseen_wake(points)

In [9]:
mesh = pv.PolyData(points)
mesh["flow"] = flow
glyph = mesh.glyph(orient="flow", scale="flow", factor=0.005)
pl = pv.Plotter()
pl.add_mesh(glyph)
pl.show_axes()
pl.show()

Widget(value='<iframe src="http://localhost:53988/index.html?ui=P_0x1746b2690_1&reconnect=auto" class="pyvista…

### 1.3 `_compute_surface_coords`

Compute evenly spaced coordinates on a surface.

In [18]:
a = 6e-4
U = np.array([0, 0.3, 0.])
bubble = Bubble(a, U=U)

In [59]:
surf_coords, normal, diff_a = bubble._compute_surface_coords()

In [66]:
mesh = pv.PolyData(surf_coords)
mesh["norm"] = normal
glyph = mesh.glyph(orient="norm", scale="norm", factor=0.0001)
pl = pv.Plotter()
mesh["z"] = surf_coords[:, 2]
pl.add_mesh(mesh, scalars="z")
pl.add_mesh(glyph)
pl.show()

Widget(value='<iframe src="http://localhost:57825/index.html?ui=P_0x14d9a5970_23&reconnect=auto" class="pyvist…

### 1.4 `_compute_surface_tangent_xy`

Compute tangent units on the sphere that are in the xy plane. 

In [ ]:
a = 6e-4
U = np.array([0, 0.3, 0.])
bubble = Bubble(a, U=U)

In [ ]:
surf_coords, _, _ = bubble._compute_surface_coords()
tangent = bubble._compute_surface_tangent_xy()

In [ ]:
mesh = pv.PolyData(surf_coords)
mesh["tangent"] = tangent
glyph = mesh.glyph(orient="tangent", scale="tangent", factor=0.0001)
pl = pv.Plotter()
mesh["z"] = surf_coords[:, 2]
pl.add_mesh(mesh, scalars="z")
pl.add_mesh(glyph)
pl.show()

Widget(value='<iframe src="http://localhost:57825/index.html?ui=P_0x14d9b7080_24&reconnect=auto" class="pyvist…

# Appendix

## A. Test velocity

In [13]:
a = 6e-4
U = np.array([0, 0.3, 0])
theta = 22.5 / 180 * np.pi
up = np.array([np.sin(theta), -np.cos(theta), 0])

im = Bubble(a, U=U)
re = Bubble(a, U=np.array([0, .1, 0]))
im.set_pos(up*a*0.01)
re.set_pos([0, 0, 0])

flow = im.Oseen_wake(re.surf_coords+re.pos)
surface_flow = (flow * re.unit_tangents).sum(axis=1, keepdims=True)

In [14]:
flow = im.Oseen_wake(re.surf_coords+re.pos)
im_grid = pv.Sphere(radius=im.a, center=im.pos)
re_grid = pv.Sphere(radius=re.a, center=re.pos)
grid = pv.PolyData(re.surf_coords+re.pos)

grid["flow"] = surface_flow * re.unit_tangents
grid["tangent"] = re.unit_tangents
glyph = grid.glyph(orient="flow", scale="flow", factor=1e-2)
tan = grid.glyph(orient="tangent", scale="tangent", factor=1e-4)
pl = pv.Plotter()
pl.add_mesh(im_grid, opacity=.5)
pl.add_mesh(re_grid)
pl.add_mesh(glyph)
pl.add_mesh(tan)
pl.camera_position = [
    (0, 0, -8*im.a),
    (0, 0, 0),
    up
]
pl.show_axes()
pl.show()

Widget(value='<iframe src="http://localhost:50264/index.html?ui=P_0x15dd49b20_2&reconnect=auto" class="pyvista…

In [7]:
circulation = 1 / 2 / re.a * surface_flow.sum() * re.ds * np.array([0, 0, 1])

In [8]:
circulation

Vector([0.00000000e+00, 0.00000000e+00, 3.40559667e-05])

In [11]:
lift = 4 * np.pi * a**3 / 3 * 1000 / np.pi / a**2 * np.cross(circulation, re.U)

In [12]:
lift

array([-2.72447733e-06,  0.00000000e+00,  0.00000000e+00])

In [28]:
from bubble_bouncing.bubble import forces

In [ ]:
forces.compute_lift(im.a, flow, im.ds, )